In [1]:
pip install --upgrade snowflake-connector-python

Note: you may need to restart the kernel to use updated packages.


In [55]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')

In [ ]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password='5eWxWv4EvyCkhkY',
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS testschema_mg")
        cs.execute("USE WAREHOUSE tiny_warehouse_mg")
        cs.execute("USE SCHEMA testschema_mg")
        runQueries(cs)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

In [241]:
def runQueries(cs):
    cs.execute(
    """CREATE OR REPLACE TABLE 
    article_table_2(id INT AUTOINCREMENT PRIMARY KEY, article_name string)""")
    data = fetch_api()
    count = int(data['totalResults'])
    articles = data['articles']
    for x in range(0,count):
        singleArticle = articles[0]
        query = "INSERT INTO article_table_2(article_name) VALUES (%s)"
        cs.execute(query.strip(),(str(singleArticle['source']['name'])))
    out = cs.fetchall()
    print(out[0])

In [135]:
def fetch_api():
    url = f"https://newsapi.org/v2/everything?q=apple&from=2025-02-05&to=2025-02-05&sortBy=popularity&apiKey=6725c58e80a8406598187079b42ed0dc"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error fetching exchange rate. Please check your input and try again.")
        return None

In [169]:
def safe_sql_string(value):
    """Wraps string values in single quotes and escapes internal single quotes."""
    return f"'{value.replace("'", "''")}'" if isinstance(value, str) else str(value)